# DINO: Self Distillation with No Labels

This notebook is the implementation for a model that is called DINO, created by the Facebook AI Research Team. DINO is a very useful model that learns image representations using contrastive learning, unlike traditional ViT, DINO is trained in a self supervised manner like Masked Autoencoders and SimCLR. The notebook compasses multiple topics like Dino, Dinov2 as well as using registers in transformers to show how they can help in the model performance.

Reference Papers: -
* [Emerging Properties in Self-Supervised Vision Transformers](https://arxiv.org/pdf/2104.14294)
* [DINOv2: Learning Robust Visual Features without Supervision](https://arxiv.org/pdf/2304.07193)
* [VISION TRANSFORMERS NEED REGISTERS](https://arxiv.org/pdf/2309.16588)

# A. DINO v1

## 1. Introduction

Dino is a methodoly proposed by researchers at Facebook AI Research, the main problem with how ViT's are traditionally trained is that they are trained using supervised datasets, that is they learn representations for images by using the loss that is created by the final MLP head which is used to predict the class the image belongs to. Although this gives rich representations it severely limits the models capability of learning as it needs a very large amount of labelled data. Transformers that are trained on text, during the pretraining stage train using a self supervised methodology as they simply try to predict the next word based on the previous n number of words in the sequence, since this does not require seperate labels, the transformer can fit on a much larger dataset, as since data does not need to be labelled there is a much larger availability of such data. 

Dino methodology aims to adress the problem by giving a method that allows the ViT to train in a self supervised manner which is called contrastive learning. Since it no longer needs labelled data, the weakness of needed labelled data no longer exists and just images themselves are enough to be given as input to the model.

## 2. Architecture

The DINO methodology as stated earlier trains in a self supervised manner that comes under Contrastive Learning. The Dino methodology actually utilises a type of training that employs two networks, a student network and a teacher network. 

Student teacher networks are a very popular method of training models in deep learning. Usually a teacher network is a large network that has been already trained on some specific task for example, lets say that the teacher network has been trained on a large dataset like ImageNet for the purpose of classifying images in that dataset, this teacher network is also very large and as a result cannot be deployed on mobile devices, so we need a smaller model. This problem is solved by using the teacher network to train a student network. This student network is a much smaller network than the teacher network, during training input is given to the student and teacher networks and the goal is to get the students softmax predictions to come as close to the teachers predictions, i.e, we are trying to get the student network to mimic the teacher network. This process is known as **knowledge distillation**. Teacher networks are usually frozen, i.e, their weights are not updated. It is impprtant to remember that distillation is NOT the same as transfer learning. During inference, the student model is what is deployed on devices and it is used for giving learned representations.

DINO as stated earlier uses such a student teacher network, however in DINO, the networks are essentially have the same architecture and the teacher is not pretrained rather it too trains during the learning process. During initialisation, the weights of the student and the teacher are kept the same, and loss is backpropogated to update student values, but the updation method for the teacher is different, we will look into this later. 

Both the student as well as teacher networks are made up of two components, a **backbone**: which is a ViT or a ResNet, a **projection head**. The features that are used in downstream tasks are the projection head outputs. The projection head is a 3 layer MLP, with a hidden layer having 2048 dimension and a l2 normalisation as well as an output layer with K dimensional output and a weight normalisation.

The model utilises two operations **centering** and **sharpening** to avoid **collapse**. We will look into these operations and what they mean when we go through the training methodology used by the DINO model.

## 3. Methodology

In this section we will go through the methodology of how the inputs are given to the DINO model and how it trains to give the best possible learned representations for an image. Earlier it was mentioned that DINO learning mechanism is called contrastive learning, and that it trains in a self-supervised manner. This is how it is able to train using self supervision. A diagram for this process is given below: 


<br>
<center><img src="https://miro.medium.com/v2/resize:fit:1400/1*huuMgEbBryxXUufW33uhvQ.png" width="80%"></center>
<br>

1. **Augmentation:** an input image x goes through an augmentation process, this can vary depending on the implementation but in essence, it is augmented to give 2 global views and n number of local views. A global view as the name suggest is a broader picture of the image, that is it isa crop of the image that has most of the image information on the other hand a local view is a crop of the image that only has a small part of the image. These views are then as per usual when working with ViT's given patch as well as positional embeddings.
2. **Teacher Network:** the teacher network gets only the two global views and then gives learned representations for these global views. The teacher outputs and centered, and its weights are updated through ewma of student (reason given in updation section).
3. **Student Network:** the student network gets the global views as well as all the local views and gives us the learned representations for these views. The student network is used during inference to get the learned representation.
4. **Contrastive Loss:** the loss for the model is calculated via the cross entropy between the teacher and student predictions. Since there are multiple samples, outputs from the teacher and student and paired up together and the loss is found for the pairs, this loss is then aggregated to find the final loss. Before loss is calculated, the teacher outputs are centered and sharpened. Centering is used because sometimes values being predicted could be very large and so it could lead to exploding values, however by subtracting the center vector which is in essence a runing mean with momentum this problem is avoided. Sharpening is used to increase values that have high probability and reduce values that have low probability. These two are used toghether to avoid collapse. **Neural Collapse**: occurs when the model starts to break down and give the same predictions no matter the output, this is something that needs to be avoided and centering and sharpening can be used to prevent this from occuring. Another important note is that a pair will never consist of two values from the same input that is if we have global view A and B then we compare student output of A and teacher of B, never student of A and teacher of A.
5. **Updation Mechanism:** the student is simply updated through backpropogation, however this is not the case with the teacher, the teacher is updated through the exponential weighted moving average (ewma) of the students parameters. This is done for stability reasons, we dont want to keep dynamically changing the weights of both student and teacher through backpropogation as in this case both student and teacher are in the benginning just noise, to ensure there is some stability to the learning process, we update teacher through the ewma of student.

The idea of DINO is that different augmented views of a image should have the same representation regardless of the augmentation, this is why we try to reduce the loss between student and teacher predictions of the global and local views. By training in such a manner, the idea is that eventually the model will get to a point and the student and teacher predictions will converge.

## 4. Implementation

In this section we will implement the DINO model from scratch and evaluate the model to see its loss.